In [76]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.api as sm

## STEP 1: Load and Explore Data

In [2]:
retail = pd.read_csv("../cohort_analysis/online_retail_II.csv")

In [3]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail.shape

(1067371, 8)

In [5]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 5942


In [6]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 34335 duplicated rows


In [7]:
#viewing duplicated rows
retail[retail.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
390,489517,84951A,S/4 PISTACHIO LOVEBIRD COASTERS,1,2009-12-01 11:34:00,2.55,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
657,489529,22028,PENNY FARTHING BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom
658,489529,22036,DINOSAUR BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom


Note: After manual review, there are indeed lots of duplicated rows (e.g. row 371 and 394). Let's remove them.

In [8]:
retail = retail.drop_duplicates(keep='first')

In [9]:
#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

Note: there are 234007 rows without Customer ID. Since we are working at the customer level, we cannot aggregate these columns. Let's remove them from further analysis.

In [10]:
retail.isna().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

In [11]:
retail = retail[retail['Customer ID'].notna()]

In [12]:
retail.describe()

,Quantity,Price,Customer ID
count,797885.000000,797885.000000,797885.000000
mean,12.602980,3.702732,15313.062777
std,191.670371,71.392549,1696.466663
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13964.000000
50%,5.000000,1.950000,15228.000000
75%,12.000000,3.750000,16788.000000
max,80995.000000,38970.000000,18287.000000


Note: there is negative quantity in the dataset. Let's check it.

In [13]:
retail[retail.Quantity < 0].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [14]:
print(f"there are {len(retail[retail.Quantity < 0].Quantity)} rows with negative quantity.")

there are 18390 rows with negative quantity.


Note: Apparently returns. The invoices of these rows also start with C, which, according to description of the dataset, means cancellations. We want to keep this information.

In [15]:
# number of unique values in each column:
for col in retail.columns:
    print(f"{col}: {retail[col].nunique()}")

Invoice: 44876
StockCode: 4646
Description: 5299
Quantity: 643
InvoiceDate: 41439
Price: 1022
Customer ID: 5942
Country: 41


In [16]:
# According to specifications Stock Code needs to be 5 digits. Check for any non digit in Stock Code
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.0,12682.0,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.0,12636.0,USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.0,12362.0,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.0,12533.0,Germany
735,C489535,D,Discount,-1,2009-12-01 12:11:00,9.0,15299.0,United Kingdom


In [17]:
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].StockCode.unique()

array(['POST', 'D', 'M', 'C2', 'BANK CHARGES', 'TEST001', 'TEST002',
       'PADS', 'ADJUST', 'ADJUST2', 'SP1002', 'DOT', 'CRUK'], dtype=object)

Note: there are many more mysterious Stock Codes than just 5 digit number. So after manual verification:<br>
'POST' = Postage <br>
'D' = Discount<br>
'M' = Manual<br>
'C2' = Carriage <br>
'BANK CHARGES' = Bank Charges <br>
'TEST001', 'TEST002' = test product  <br>
'PADS' = Pads to match all cushions <br>
'ADJUST' = Adjustment by john on 26/01/2010 16 <br>
'ADJUST2' = Adjustment by Peter on Jun 25 2010 <br>
'SP1002' = KID'S CHALKBOARD/EASEL <br>
'DOT' = DOTCOM postage <br>
'CRUK' = CRUK commission <br>

Apparently, we can exclude the following categories 'POST', 'C2', 'DOT', 'CRUK' and 'BANK CHARGES' because these are expenses for the company and they do not reflect the actual revenue streams. 
I decided to live other categories because they are either related to the products (test product, pads or SP1002) or revenue streams (discounts, adjustments, manual)

In [18]:
# excluding all the undesirable StockCode values 
exclude_test_idx = retail[retail.StockCode.str.contains("POST|C2|DOT|CRUK|BANK CHARGES", na=False, case=False)].index
retail = retail[~retail.index.isin(exclude_test_idx)]

In [19]:
#checking results
retail[~retail.StockCode.str.match(pat='[0-9]{5}')].StockCode.unique()

array(['D', 'M', 'TEST001', 'TEST002', 'PADS', 'ADJUST', 'ADJUST2',
       'SP1002'], dtype=object)

In [20]:
# checking types of columns
retail.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [21]:
retail.InvoiceDate = pd.to_datetime(retail.InvoiceDate.astype(str), format="%Y-%m-%d %H:%M:%S")

## Step 2: Feature Engineering

In [22]:
# adding column with Invoice Month
retail['InvoiceMonth'] = retail['InvoiceDate'].dt.strftime("%Y-%m")
retail['InvoiceMonth'] = pd.to_datetime(retail['InvoiceMonth'])

In [23]:
# adding column with Purchase Sum
retail['PurchaseSum'] = retail['Quantity']*retail['Price']

In [24]:
#explore sales distribution by month
retail.groupby(['InvoiceMonth']).size()

InvoiceMonth
2009-12-01    31217
2010-01-01    22079
2010-02-01    23515
2010-03-01    32519
2010-04-01    27363
2010-05-01    29135
2010-06-01    31378
2010-07-01    27280
2010-08-01    26514
2010-09-01    34813
2010-10-01    49594
2010-11-01    59942
2010-12-01    26275
2011-01-01    21586
2011-02-01    20068
2011-03-01    27413
2011-04-01    22920
2011-05-01    28553
2011-06-01    27478
2011-07-01    27170
2011-08-01    27339
2011-09-01    40309
2011-10-01    49778
2011-11-01    64016
2011-12-01    17325
dtype: int64

Note: let's use the last available month - Dec 2011 - as the period for prediction.

In [25]:
# exclude target variable from data
X_retail = retail[retail['InvoiceMonth'] != '2011-12-01']

In [26]:
now = dt.datetime(2011, 12, 1)
now

datetime.datetime(2011, 12, 1, 0, 0)

In [184]:
features = X_retail.groupby('Customer ID').agg({'InvoiceDate': lambda x: (now - x.max()).days,
                                               'Invoice': 'nunique',
                                               'PurchaseSum': ['sum', 'mean'],
                                               'Quantity': ['mean', 'sum']}).reset_index()

In [185]:
features.columns = ['CustomerID', 'Recency', 'Frequency', 'MonetaryValue', 'PurchaseSize_avg', 'Quantity_avg', 'Quantity_total']
features.head()

,CustomerID,Recency,Frequency,MonetaryValue,PurchaseSize_avg,Quantity_avg,Quantity_total
0,12346.0,316,17,-51.74,-1.100851,1.127660,53
1,12347.0,30,7,4696.71,22.259289,13.151659,2775
2,12348.0,66,5,1658.40,36.052174,58.782609,2704
3,12349.0,9,4,3654.54,20.647119,9.129944,1616
4,12350.0,301,1,294.40,18.400000,12.250000,196


In [186]:
print(f"there are {features.shape[0]} customers for whom we will predict transactions in Dec 2011")
print(f"there are {retail['Customer ID'].nunique()} customers in total")

there are 5912 customers for whom we will predict transactions in Dec 2011
there are 5940 customers in total


Note: Apparently, we acquired 28 new customers in Dec 2011. We will need to exclude these from our target variable.

## Step 3: Calculate Target Variable

In [187]:
# build pivot table with monthly transactions per customer
customer_monthly_transactions = pd.pivot_table(data=retail, index='Customer ID', columns='InvoiceMonth', 
                                         values='Invoice', aggfunc='nunique', fill_value=0)

In [188]:
customer_monthly_transactions.head()

InvoiceMonth,2009-12-01,2010-01-01,2010-02-01,2010-03-01,2010-04-01,2010-05-01,2010-06-01,2010-07-01,2010-08-01,2010-09-01,...,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,2011-10-01,2011-11-01,2011-12-01
Customer ID,,,,,,,,,,,,,,,,,,,,,
12346.0,5,5,0,1,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,1
12348.0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
12349.0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
# store identifier and target variable column names
custid = ['CustomerID']
target = '2011-12-01'

In [190]:
y = customer_monthly_transactions[target]
y.head()

Customer ID
12346.0    0
12347.0    1
12348.0    0
12349.0    0
12350.0    0
Name: 2011-12-01 00:00:00, dtype: int64

In [191]:
# comparing the size of target and feature variables
y.index.size, features['CustomerID'].values.size

(5940, 5912)

In [192]:
# we need to exclude customers acquired in Dec 2011
y = y[y.index.isin(features['CustomerID'].values)]

In [193]:
# extract feature column names
cols = [col for col in features.columns if col not in custid]
cols

['Recency',
 'Frequency',
 'MonetaryValue',
 'PurchaseSize_avg',
 'Quantity_avg',
 'Quantity_total']

In [194]:
X = features[cols]

## Step 4: Modeling with Linear Regression

In [195]:
# randomly split 25% of data to testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=121)

In [196]:
# Initialize the model
linreg = LinearRegression()
# Fit the model
linreg.fit(X_train, y_train)

# Predict on training and testing data
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

In [197]:
# Calculate metrics for training data
rmse_train = np.sqrt(mean_squared_error(y_train_pred, y_train)).round(4)
mae_train = mean_absolute_error(y_train_pred, y_train).round(4)
rmse_train, mae_train

(0.4285, 0.2225)

In [198]:
# Calculate metrics for testing data
rmse_test = np.sqrt(mean_squared_error(y_test_pred, y_test)).round(4)
mae_test = mean_absolute_error(y_test_pred, y_test).round(4)
rmse_test, mae_test

(0.4433, 0.224)

Note: the errors are slightly higher for the test data, which is expected, because we trained our data to build the model and so it matched the patterns in training data better than in the unseen test data. <br>

MAE of 0.22 means that our model is off by 0.22 transaction when comparing actual transactions in December 2011 with our predicted transactions in December 2011.

# Step 5: Interpreting Coefficients

We will assess statistical significance of the model coefficients using statsmodels library.

In [199]:
#convert target variable to numpy array
y_train = np.array(y_train)

In [200]:
# Initialize the model
olsreg = sm.OLS(y_train, X_train)
olsreg = olsreg.fit()

In [201]:
olsreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.324
Model:                            OLS   Adj. R-squared (uncentered):              0.323
Method:                 Least Squares   F-statistic:                              353.5
Date:                Thu, 09 Apr 2020   Prob (F-statistic):                        0.00
Time:                        12:37:38   Log-Likelihood:                         -2550.2
No. Observations:                4434   AIC:                                      5112.
Df Residuals:                    4428   BIC:                                      5151.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Recency          -8.696e-05   2.27e-05     -3.835      0.000      -0.000   -4.25e-05
Frequency            0.0218      0.001     34.874      0.000       0.021       0.023
MonetaryValue    -9.325e-06   1.73e-06     -5.380      0.000   -1.27e-05   -5.93e-06
PurchaseSize_avg  1.721e-05   1.63e-05      1.058      0.290   -1.47e-05    4.91e-05
Quantity_avg      1.818e-06   3.34e-05      0.054      0.957   -6.37e-05    6.73e-05
Quantity_total    3.677e-06   1.83e-06      2.011      0.044    9.28e-08    7.26e-06
==============================================================================
Omnibus:                     3485.738   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           130763.134
Skew:                           3.438   Prob(JB):                         0.00
Kurtosis:                      28.700   Cond. No.                     1.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Note:** our model explains only 32% of variance (R-squared=0.324). This is relatively low, meaning that the model poorly fits the variation in the target variable.<br>
The first column (coef) in the summary table shows the coefficients that can be interpreted as the change in the target variable given one unit increase in the feature. 
    E.g. frequency=0.0218 means that customer that customer who's frequency is higher by 1 (invoice) in the pre-December period will on average have 0.0218 invoices more in December 2011. <br>
    
Given the statistical signficance level of 95%, we have only three coefficients - recency, frequency, monetary value - that are statistically significant.

In [202]:
tx_in_Dec_pred = linreg.predict(X)

In [204]:
features['predicted_tx'] = pd.Series(np.round(tx_in_Dec_pred, 3))

In [205]:
y_fin = y.reset_index()
y_fin.columns = ['Customer ID', 'actual_tx']

In [207]:
retail_predicted = features.merge(y_fin, how='inner', left_on='CustomerID', right_on='Customer ID').drop('Customer ID', axis=1)

In [209]:
retail_predicted[retail_predicted.actual_tx == 0]

,CustomerID,Recency,Frequency,MonetaryValue,PurchaseSize_avg,Quantity_avg,Quantity_total,predicted_tx,actual_tx
0,12346.0,316,17,-5.174000e+01,-1.100851e+00,1.127660,53,0.334,0
2,12348.0,66,5,1.658400e+03,3.605217e+01,58.782609,2704,0.142,0
3,12349.0,9,4,3.654540e+03,2.064712e+01,9.129944,1616,0.114,0
4,12350.0,301,1,2.944000e+02,1.840000e+01,12.250000,196,0.013,0
5,12351.0,366,1,3.009300e+02,1.433000e+01,12.428571,261,-0.001,0
6,12352.0,27,13,1.609210e+03,1.490009e+01,6.027778,651,0.305,0
7,12353.0,195,2,4.067600e+02,1.694833e+01,8.833333,212,0.055,0
8,12354.0,223,1,1.079400e+03,1.861034e+01,9.137931,530,0.025,0
9,12355.0,205,2,9.476100e+02,2.707457e+01,15.514286,543,0.050,0
10,12356.0,13,6,5.611730e+03,4.037216e+01,24.287770,3376,0.143,0


## Acknowledgement:

**Data Source:** Online Retail II Data Set, UCI Machine Learning Repository, http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Analysis done after completing the course "Customer Segmentation in Python" on DataCamp (instructor - 
Karolis Urbonas)